# Загрузка Pandas и очистка данных

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
import re

In [2]:
# class fillnan():
    

1. ~Создать признаки для Price Range.~
2. ~Создать признаки для населения по городам и площади городов.~
3. Спарсить средний чек.
4. ~Посмотреть ***Ranking — место, которое занимает данный ресторан среди всех ресторанов своего города***, ***Number of Reviews — количество отзывов о ресторане***.~
5. Попробывать спарсить расстояние от центра города до ресторана. На https://www.tripadvisor.ru/ есть координаты ресторанов, ~на https://www.google.com/maps/ парсятся центры городов~.
6. Спрарсить страны на Яндекс.Картах

In [3]:
df = pd.read_csv('main_task.csv')

# очистка столбца виды кухонь
pattern = re.compile('\w+\s*\w*')
df['Cuisine Style'].fillna('no_information', inplace=True)
df['Cuisine Style'] = df['Cuisine Style'].apply(lambda x: pattern.findall(x))

#создание столбца Cuisine Count - колическтво кухонь в ресторане
df['Cuisine Count'] = df['Cuisine Style'].apply(lambda x: len(x))

# очистка столбца с отзывами
pattern = re.compile('\d+[-]\d+[-]\d+')
df['Reviews'] = df['Reviews'].apply(lambda x: str(x).split(','))
df['Reviews'] = df['Reviews'].apply(lambda x: str(x).replace('/', '-'))
df['Reviews'] = df['Reviews'].apply(lambda x: pattern.findall(x))

df['Reviews'] = df['Reviews'].apply(lambda x: pd.to_datetime(x))
df['Reviews'] = df['Reviews'].apply(lambda x: np.nan if len(x) == 0 else x)

#создание столбца diff_day - разница между отзывами в днях
df['dif_day'] = df['Reviews'][df['Reviews'].isnull() == False].apply(lambda x: x[0] - x[-1])
df['dif_day'] = df['dif_day'].apply(lambda x: x.days)

df['City'][df['City'] == 'Oporto'] = 'Porto'

<ipython-input-3-305776c682f1>:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['City'][df['City'] == 'Oporto'] = 'Porto'


In [4]:

df['Price Range'] = df['Price Range'].apply(lambda x: 'low_price' if x == '$' else x)
df['Price Range'] = df['Price Range'].apply(lambda x: 'medium_price' if x == '$$ - $$$' else x)
df['Price Range'] = df['Price Range'].apply(lambda x: 'high_price' if x == '$$$$' else x)

In [5]:
#борьба с нулевыми значениями
df['Number of Reviews'] = df['Number of Reviews'].fillna(df['Number of Reviews'].median())
df['dif_day'] = df['dif_day'].fillna(df['dif_day'].median())

In [6]:
df.head(2)

,Restaurant_id,City,Cuisine Style,Ranking,Rating,Price Range,Number of Reviews,Reviews,URL_TA,ID_TA,Cuisine Count,dif_day
0,id_5569,Paris,"[European, French, International]",5570.0,3.5,medium_price,194.0,"DatetimeIndex(['2017-12-31', '2017-11-20'], dt...",/Restaurant_Review-g187147-d1912643-Reviews-R_...,d1912643,3,41.0
1,id_1535,Stockholm,[no_information],1537.0,4.0,NaN,10.0,"DatetimeIndex(['2017-07-06', '2016-06-19'], dt...",/Restaurant_Review-g189852-d7992032-Reviews-Bu...,d7992032,1,382.0


In [7]:
# df['Cuisine Style'].unique()

In [8]:
#Создание Dummy - переменных
df = df.merge(pd.get_dummies(df['Price Range']), left_index=True, right_index=True)
df = df.merge(pd.get_dummies(df['City']), left_index=True, right_index=True)

In [9]:
#создаём признаки по населению и площади городов
data = [[2.2, 105], [1, 188], [8.9, 1706], 
         [3.6, 891], [1.5, 310], [0.3, 41,4], 
         [1.4, 181], [0.4, 368], [1.9, 414], 
         [2.9, 1287], [1.7, 101], [3.3, 607], 
         [1.2, 117], [0.2, 32.6], [0.4, 88], 
         [1.8, 517], [1.8, 525], [0.8, 86.4], 
         [0.9, 219], [0.5, 48], [1.9, 755], 
         [0.5,100], [1.3, 496], [0.7, 454], 
         [0.7,213], [0.5, 120], [0.2, 16], 
         [0.3, 163], [0.7, 39], [0.125, 51.5], 
         [0.8, 327]]
cities = df.City.unique()
city_dict = {}
for i in range(len(cities)):
    city_dict[cities[i]] = data[i] 

df['city_pop'] = df['City'].apply(lambda x: city_dict[x][0])
df['city_val'] = df['City'].apply(lambda x: city_dict[x][-1])

# Разбиваем датафрейм на части, необходимые для обучения и тестирования модели

In [10]:
# Х - данные с информацией о ресторанах, у - целевая переменная (рейтинги ресторанов)
X = df.drop(['Restaurant_id', 'Rating', 'Cuisine Style', 'City',
             'Price Range', 'Reviews', 'URL_TA', 'ID_TA'], axis=1)
y = df['Rating']

In [11]:
X

,Ranking,Number of Reviews,Cuisine Count,dif_day,high_price,low_price,medium_price,Amsterdam,Athens,Barcelona,...,Paris,Porto,Prague,Rome,Stockholm,Vienna,Warsaw,Zurich,city_pop,city_val
0,5570.0,194.0,3,41.0,0,0,1,0,0,0,...,1,0,0,0,0,0,0,0,2.2,105.0
1,1537.0,10.0,1,382.0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,1.0,188.0
2,353.0,688.0,8,2.0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,8.9,1706.0
3,3458.0,3.0,1,38.0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,3.6,891.0
4,621.0,84.0,3,272.0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1.5,310.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39995,500.0,79.0,5,34.0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1.4,181.0
39996,6341.0,542.0,5,9.0,0,0,1,0,0,0,...,1,0,0,0,0,0,0,0,2.2,105.0
39997,1652.0,4.0,2,3127.0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,1.0,188.0
39998,641.0,70.0,5,23.0,0,0,1,0,0,0,...,0,0,0,0,0,0,1,0,1.8,517.0


In [12]:
# Загружаем специальный инструмент для разбивки:
from sklearn.model_selection import train_test_split

In [13]:
# Наборы данных с меткой "train" будут использоваться для обучения модели, "test" - для тестирования.
# Для тестирования мы будем использовать 25% от исходного датасета.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

# Создаём, обучаем и тестируем модель

In [14]:
# Импортируем необходимые библиотеки:
from sklearn.ensemble import RandomForestRegressor # инструмент для создания и обучения модели
from sklearn import metrics # инструменты для оценки точности модели

In [15]:
# Создаём модель
regr = RandomForestRegressor(n_estimators=100)

# Обучаем модель на тестовом наборе данных
regr.fit(X_train, y_train)

# Используем обученную модель для предсказания рейтинга ресторанов в тестовой выборке.
# Предсказанные значения записываем в переменную y_pred
y_pred = regr.predict(X_test)

In [16]:
# Сравниваем предсказанные значения (y_pred) с реальными (y_test), и смотрим насколько они в среднем отличаются
# Метрика называется Mean Absolute Error (MAE) и показывает среднее отклонение предсказанных значений от фактических.
print('MAE:', round(metrics.mean_absolute_error(y_test, y_pred),3))

#0.38
#0.21

MAE: 0.209
